In [ ]:
# default_exp preprocess_data
# default_cls_lvl 2

In [ ]:
%load_ext autotime

# Data preparation

> Dieses Modul werden die Daten runtergeladen und für die Verarbeitung vorbereitet.

Die Datenbereinigung halten wir möglichst einfach, denn unsere Ziel ist es zu verstehen ob eine Kollaboration mit Modellen einen Vorteil bringt. Deshalb beschränken wir uns bei der Datenbereinigung auf folgende Punkte:
 - alle NANs werden entfernt und durch Integer ersetzt
 - alle Characters werden mit Integer codiert
 
Die meisten Ideen haben wir von [jturkewitz](https://github.com/jturkewitz/SideProjects/tree/master/Kaggle/Santander_Prod) ausgeliehen.
 

In [ ]:
#hide
from nbdev.showdoc import *

time: 442 ms


## Download und unzip Data

In [ ]:
#export
import os
import gdown
import pandas as pd
import numpy as np
import time
from fastscript import *
from nbdev import export
from pathlib import Path
from zipfile import ZipFile

time: 292 ms


In [ ]:
#export 
def download_santander_data():
    """downloads the data from gdrive to the data folder"""   
    dest = Path('data/raw/train_ver2.csv.zip')
    
    #only download if file not already exists
    if not dest.exists(): 
        dest.parent.mkdir(parents=True, exist_ok=True)
        
        gdown.download(
            'https://drive.google.com/uc?export=download&id=1H-dFcuqI99OkXAawsTMkmylBz3exNUgZ', 
            'data/raw/train_ver2.csv.zip',
            quiet=False)
    else:
        print(f"file {dest} already exists.")
        
    return dest

time: 661 µs


In [ ]:
path_file = download_santander_data()

file data/raw/train_ver2.csv.zip already exists.
time: 4.39 ms


In [ ]:
#export 
def unzip(path:[Path,str], dest:[Path,str]='.'):
    """unzips a given file"""
    path = Path(path)
    dest = path.parent / path.stem
    
    if not dest.exists():
        with ZipFile(str(path), 'r') as zipObj:
            zipObj.extractall(str(path.parent))
            print(f'extracted to {path.parent / path.stem}')
    else:
        print(f"file {dest} already exists!")
    
    return dest

time: 2.43 ms


In [ ]:
#export
def timeit(method):
    """timeit decorator to measure execution times"""
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

time: 2.26 ms


In [ ]:
path_file = unzip(path_file, path_file.parent)

file data/raw/train_ver2.csv already exists!
time: 2.08 ms


## Clean data

Hierfür verwenden wir im Wesentlichen die Empfehlungen von [Alan Pryors Jupyter Notebook](https://www.kaggle.com/apryor6/detailed-cleaning-visualization-python).

### Dictonaries für die Codierung

In [ ]:
#export
canal_dict = {'KAI': 35,'KBG': 17,'KGU': 149,'KDE': 47,'KAJ': 41,'KCG': 59,
 'KHM': 12,'KAL': 74,'KFH': 140,'KCT': 112,'KBJ': 133,'KBL': 88,'KHQ': 157,'KFB': 146,'KFV': 48,'KFC': 4,
 'KCK': 52,'KAN': 110,'KES': 68,'KCB': 78,'KBS': 118,'KDP': 103,'KDD': 113,'KBX': 116,'KCM': 82,
 'KAE': 30,'KAB': 28,'KFG': 27,'KDA': 63,'KBV': 100,'KBD': 109,'KBW': 114,'KGN': 11,
 'KCP': 129,'KAK': 51,'KAR': 32,'KHK': 10,'KDS': 124,'KEY': 93,'KFU': 36,'KBY': 111,
 'KEK': 145,'KCX': 120,'KDQ': 80,'K00': 50,'KCC': 29,'KCN': 81,'KDZ': 99,'KDR': 56,
 'KBE': 119,'KFN': 42,'KEC': 66,'KDM': 130,'KBP': 121,'KAU': 142,'KDU': 79,
 'KCH': 84,'KHF': 19,'KCR': 153,'KBH': 90,'KEA': 89,'KEM': 155,'KGY': 44,'KBM': 135,
 'KEW': 98,'KDB': 117,'KHD': 2,'RED': 8,'KBN': 122,'KDY': 61,'KDI': 150,'KEU': 72,
 'KCA': 73,'KAH': 31,'KAO': 94,'KAZ': 7,'004': 83,'KEJ': 95,'KBQ': 62,'KEZ': 108,
 'KCI': 65,'KGW': 147,'KFJ': 33,'KCF': 105,'KFT': 92,'KED': 143,'KAT': 5,'KDL': 158,
 'KFA': 3,'KCO': 104,'KEO': 96,'KBZ': 67,'KHA': 22,'KDX': 69,'KDO': 60,'KAF': 23,'KAW': 76,
 'KAG': 26,'KAM': 107,'KEL': 125,'KEH': 15,'KAQ': 37,'KFD': 25,'KEQ': 138,'KEN': 137,
 'KFS': 38,'KBB': 131,'KCE': 86,'KAP': 46,'KAC': 57,'KBO': 64,'KHR': 161,'KFF': 45,
 'KEE': 152,'KHL': 0,'007': 71,'KDG': 126,'025': 159,'KGX': 24,'KEI': 97,'KBF': 102,
 'KEG': 136,'KFP': 40,'KDF': 127,'KCJ': 156,'KFR': 144,'KDW': 132,-1: 6,'KAD': 16,
 'KBU': 55,'KCU': 115,'KAA': 39,'KEF': 128,'KAY': 54,'KGC': 18,'KAV': 139,'KDN': 151,
 'KCV': 106,'KCL': 53,'013': 49,'KDV': 91,'KFE': 148,'KCQ': 154,'KDH': 14,'KHN': 21,
 'KDT': 58,'KBR': 101,'KEB': 123,'KAS': 70,'KCD': 85,'KFL': 34,'KCS': 77,'KHO': 13,
 'KEV': 87,'KHE': 1,'KHC': 9,'KFK': 20,'KDC': 75,'KFM': 141,'KHP': 160,'KHS': 162,
 'KFI': 134,'KGV': 43}


pais_dict = {'LV': 102,'CA': 2,'GB': 9,'EC': 19,'BY': 64,'ML': 104,'MT': 118,
 'LU': 59,'GR': 39,'NI': 33,'BZ': 113,'QA': 58,'DE': 10,'AU': 63,'IN': 31,
 'GN': 98,'KE': 65,'HN': 22,'JM': 116,'SV': 53,'TH': 79,'IE': 5,'TN': 85,
 'PH': 91,'ET': 54,'AR': 13,'KR': 87,'GA': 45,'FR': 8,'SG': 66,'LB': 81,
 'MA': 38,'NZ': 93,'SK': 69,'CN': 28,'GI': 96,'PY': 51,'SA': 56,'PL': 30,
 'PE': 20,'GE': 78,'HR': 67,'CD': 112,'MM': 94,'MR': 48,'NG': 83,'HU': 106,
 'AO': 71,'NL': 7,'GM': 110,'DJ': 115,'ZA': 75,'OM': 100,'LT': 103,'MZ': 27,
 'VE': 14,'EE': 52,'CF': 109,'CL': 4,'SL': 97,'DO': 11,'PT': 26,'ES': 0,
 'CZ': 36,'AD': 35,'RO': 41,'TW': 29,'BA': 61,'IS': 107,'AT': 6,'ZW': 114,
 'TR': 70,'CO': 21,'PK': 84,'SE': 24,'AL': 25,'CU': 72,'UY': 77,'EG': 74,'CR': 32,
 'GQ': 73,'MK': 105,'KW': 92,'GT': 44,'CM': 55,'SN': 47,'KZ': 111,'DK': 76,
 'LY': 108,'AE': 37,'PA': 60,'UA': 49,'GW': 99,'TG': 86,'MX': 16,'KH': 95,
 'FI': 23,'NO': 46,'IT': 18,'GH': 88, 'JP': 82,'RU': 43,'PR': 40,'RS': 89,
 'DZ': 80,'MD': 68,-1: 1,'BG': 50,'CI': 57,'IL': 42,'VN': 90,'CH': 3,'US': 15,'HK': 34,
 'CG': 101,'BO': 62,'BR': 17,'BE': 12,'BM': 117}


emp_dict = {'N':0,-1:-1,'A':1,'B':2,'F':3,'S':4}
indfall_dict = {'N':0,-1:-1,'S':1}
sexo_dict = {'V':0,'H':1,-1:-1}
tiprel_dict = {'A':0,-1:-1,'I':1,'P':2,'N':3,'R':4}
indresi_dict = {'N':0,-1:-1,'S':1}
indext_dict = {'N':0,-1:-1,'S':1}
conyuemp_dict = {'N':0,-1:-1,'S':1}
segmento_dict = {-1:-1,'01 - TOP':1,'02 - PARTICULARES':2,'03 - UNIVERSITARIO':3}

time: 8.18 ms


In [ ]:
#export
def resize_data(DF,is_DF=True):
    """codiert die Daten und ersetzt Characters und Nans mit Integers"""
    DF.replace(' NA', -1, inplace=True)
    DF.replace('         NA', -1, inplace=True)
    DF.fillna(-1, inplace=True)
    DF['ncodpers'] = DF['ncodpers'].astype(np.int32)
    DF.rename(columns={'ncodpers':'id'},inplace=True)
    DF['renta'] = DF['renta'].astype(np.float64)
    DF['renta'] = DF['renta'].astype(np.int64)
    DF['indrel'] = DF['indrel'].map(lambda x: 2 if x == 99 else x).astype(np.int8)

    DF['ind_empleado'] = DF['ind_empleado'].map(lambda x: emp_dict[x]).astype(np.int8)

    DF['sexo'] = DF['sexo'].map(lambda x: sexo_dict[x]).astype(np.int8)
    DF['age'] = DF['age'].astype(np.int16)
    DF['ind_nuevo'] = DF['ind_nuevo'].astype(np.int8)
    DF['antiguedad'] = DF['antiguedad'].map(lambda x: -1 if x == '     NA' else x).astype(int)
    DF['antiguedad'] = DF['antiguedad'].map(lambda x: -1 if x == -999999 else x).astype(np.int16)
    DF['indrel_1mes'] = DF['indrel_1mes'].map(lambda x: -2 if x == 'P' else x).astype(np.float16)
    DF['indrel_1mes'] = DF['indrel_1mes'].astype(np.int8)

    DF['tiprel_1mes'] = DF['tiprel_1mes'].map(lambda x: tiprel_dict[x]).astype(np.int8)

    DF['indresi'] = DF['indresi'].map(lambda x: indresi_dict[x]).astype(np.int8)

    DF['indext'] = DF['indext'].map(lambda x: indext_dict[x]).astype(np.int8)

    DF['conyuemp'] = DF['conyuemp'].map(lambda x: conyuemp_dict[x]).astype(np.int8)

    DF['canal_entrada'] = DF['canal_entrada'].map(lambda x: canal_dict[x]).astype(np.int16)


    DF['indfall'] = DF['indfall'].map(lambda x: indfall_dict[x]).astype(np.int8)

    DF['pais_residencia'] = DF['pais_residencia'].map(lambda x: pais_dict[x]).astype(np.int8)

    DF['tipodom'] = DF['tipodom'].astype(np.int8)

    DF['cod_prov'] = DF['cod_prov'].astype(np.int8)

    DF.drop('nomprov',axis=1,inplace=True)

    DF['ind_actividad_cliente'] = DF['ind_actividad_cliente'].astype(np.int8)

    DF['fecha_dato_month'] = DF['fecha_dato'].map(lambda x: int(x[5:7])).astype(np.int8)
    DF['fecha_dato_year'] = DF['fecha_dato'].map(lambda x: int(x[0:4]) - 2015).astype(np.int8)
    DF['month_int'] = (DF['fecha_dato_month'] + 12 * DF['fecha_dato_year']).astype(np.int8)
    DF.drop('fecha_dato',axis=1,inplace=True)

    DF['fecha_alta'] = DF['fecha_alta'].map(lambda x: '2020-01-01' if x == -1 else x)
    DF['fecha_alta_month'] = DF['fecha_alta'].map(lambda x: int(x[5:7])).astype(np.int16)
    DF['fecha_alta_year'] = DF['fecha_alta'].map(lambda x: int(x[0:4]) - 1995).astype(np.int16)
    DF['fecha_alta_day'] = DF['fecha_alta'].map(lambda x: int(x[8:10])).astype(np.int16)
    DF['fecha_alta_month_int'] = (DF['fecha_alta_month'] + 12 * DF['fecha_alta_year']).astype(np.int16)
    DF['fecha_alta_day_int'] = (DF['fecha_alta_day'] + 30 * DF['fecha_alta_month'] + 365 * DF['fecha_alta_year']).astype(np.int32)
    DF.drop('fecha_alta',axis=1,inplace=True)
    DF['ult_fec_cli_1t'] = DF['ult_fec_cli_1t'].map(lambda x: '2020-01-01' if x == -1 else x)
    DF['ult_fec_cli_1t_month'] = DF['ult_fec_cli_1t'].map(lambda x: int(x[5:7])).astype(np.int16)
    DF['ult_fec_cli_1t_year'] = DF['ult_fec_cli_1t'].map(lambda x: int(x[0:4]) - 2015).astype(np.int16)
    DF['ult_fec_cli_1t_day'] = DF['ult_fec_cli_1t'].map(lambda x: int(x[8:10])).astype(np.int16)
    DF['ult_fec_cli_1t_month_int'] = (DF['ult_fec_cli_1t_month'] + 12 * DF['ult_fec_cli_1t_year']).astype(np.int8)
    DF.drop('ult_fec_cli_1t',axis=1,inplace=True)

    DF['segmento'] = DF['segmento'].map(lambda x: segmento_dict[x]).astype(np.int8)
    

    for col in target_cols:
        if is_DF:
            DF[col] = DF[col].astype(np.int8)

    return DF


time: 7.85 ms


In [ ]:
#export 
@call_parse
def preprocess_csv(path:Param("path to the raw csv file", str) = 'data/raw/train_ver2.csv', 
                   path_save:Param("path to the destination csv file", str) = 'data/interim/01_train.csv'):
    path = Path(path)
    path_save = Path(path_save)
    reader = pd.read_csv(path, chunksize=100000, header=0)
    train = pd.concat([resize_data(chunk) for chunk in reader])
    train.to_csv(path_save, index=False)
    return train


time: 88.2 ms


In [ ]:
train = preprocess_csv()

/home/wilhelm/miniconda3/envs/CommunityLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/wilhelm/miniconda3/envs/CommunityLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (10,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/wilhelm/miniconda3/envs/CommunityLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/wilhelm/miniconda3/envs/CommunityLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or se

time: 3min 36s


In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_data_prep.ipynb.
Converted 00_data_preparation.ipynb.
Converted 00_data_preparation_CCA.ipynb.
Converted 01_data_preprocess.ipynb.
Converted 02_data_Cleaning.ipynb.
Converted 03_target_vars.ipynb.
Converted 04_base_model.ipynb.
Converted index.ipynb.
time: 147 ms
